In [ ]:
# Installation des bibliothèques nécessaires
!pip install -q langchain transformers sentence-transformers faiss-cpu accelerate torch langchain_community

# Si vous avez utilisé ChromaDB, vous pouvez l'installer, mais FAISS est souvent plus simple dans Colab
# !pip install -q chromadb

# Téléchargement des dépendances Python
import os
import torch
from langchain_community.vectorstores import FAISS # Utilisation de FAISS, plus léger que Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain.llms import HuggingFacePipeline # Le LLM de Colab
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("Installations et imports terminés.")

In [ ]:
# --- CONFIGURATION DES COMPOSANTS RAG ---

FILE_PATH = "documentation_interne.txt"

# 1. Chargement et Division des données (Chunking)
try:
    with open(FILE_PATH, 'r', encoding='utf-8') as f:
        document_content = f.read()
except FileNotFoundError:
    print(f"ERREUR: Le fichier {FILE_PATH} n'a pas été trouvé. Veuillez le téléverser dans Colab.")
    exit()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)
docs = text_splitter.create_documents([document_content])
print(f"Document divisé en {len(docs)} segments (chunks).")

# 2. EMBEDDING (Hugging Face Local et Gratuit)
# On utilise les embeddings HFE pour encoder les segments
embedding_function = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'} 
)

# 3. CRÉATION DU VECTOR STORE (Mémoire RAG)
# FAISS est le plus rapide à initialiser dans Colab
vector_store = FAISS.from_documents(docs, embedding_function)
rag_retriever = vector_store.as_retriever(search_kwargs={"k": 3})
print("✅ Base de données FAISS (RAG) créée avec succès.")

In [ ]:
# --- CONFIGURATION DU LLM (Mistral sur GPU Colab) ---

# Charger Mistral 7B Instruct depuis Hugging Face
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Utiliser le pipeline Transformers (qui utilise le GPU de Colab)
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    # Correction : On retire l'argument 'device' du pipeline_kwargs
    pipeline_kwargs={
        "max_new_tokens": 512,
        # 'device' est retiré ici pour éviter le conflit
    },
    # Et on utilise l'argument 'device' natif de la méthode LangChain si nécessaire,
    # mais souvent, LangChain le gère automatiquement si un GPU est présent.
    # Pour garantir l'utilisation du GPU T4, on passe device=0
    device=0, # <-- Ajout de device=0 pour cibler explicitement le premier GPU
    model_kwargs={"torch_dtype": torch.bfloat16} 
)
print(f"✅ LLM Agent (Mistral-7B-Instruct) chargé depuis Hugging Face sur Colab.")


# --- CONSTRUCTION DE LA CHAÎNE RAG DIRECTE (LCEL) ---

RAG_PROMPT = ChatPromptTemplate.from_template("""
Tu es un assistant IA professionnel. Utilise UNIQUEMENT le contexte fourni ci-dessous pour répondre à la question. 
Si la réponse n'est pas dans le contexte, tu dois répondre de manière polie : 'Je suis désolé, cette information spécifique n'est pas disponible dans ma documentation interne.'

--- CONTEXTE ---
{context}
--- FIN CONTEXTE ---

Question: {question}
""")

# Assemblage de la Chaîne RAG LCEL
rag_chain = (
    RunnablePassthrough.assign(
        context= (lambda x: x['question']) | rag_retriever 
    )
    | RAG_PROMPT
    | llm # Le LLM HuggingFacePipeline est utilisé ici
    | StrOutputParser()
)

# --- TESTS DE VALIDATION ---

print("\n" + "="*50)
print("TEST DE LA CHAÎNE RAG DIRECTE SUR COLAB (Validation Projet 2)")
print("="*50)

# Test 1: Question Interne (DOIT utiliser le RAG pour répondre)
print("\n--- TEST 1: Question Interne ---")
question_interne = "Combien de temps faut-il pour finaliser le traitement d'un remboursement ?"
print(f"Question : {question_interne}")
response_interne = rag_chain.invoke({"question": question_interne})
print(f"\nRÉPONSE RAG : {response_interne}")

# Test 2: Question Généraliste (DOIT répondre 'Information non disponible')
print("\n--- TEST 2: Question Générale ---")
question_generale = "Quel est le plus grand désert du monde ?"
print(f"Question : {question_generale}")
response_generale = rag_chain.invoke({"question": question_generale})
print(f"RÉPONSE RAG : {response_generale}")